In [ ]:
import numpy as np
import pycolmap
import torch
import torch.nn as nn

from splat.gaussians import Gaussians
from splat.read_utils.read_gs_ply_files import convert_to_gaussian_schema, read_ply_file
from splat.utils import read_images_binary, read_images_text

stem = "/Users/derek/Desktop/intro_to_gaussian_splatting"
# stem = "/home/da2986/intro_to_gaussian_splatting"

models_path = "/Users/derek/Downloads/models/"
ply_path = f"{models_path}/treehill/point_cloud/iteration_7000/point_cloud.ply"
vertices = read_ply_file(ply_path)
gaussians = convert_to_gaussian_schema(vertices)

colmap_path = f"{stem}/data/treehill/sparse/0"
reconstruction = pycolmap.Reconstruction(colmap_path)

points3d = reconstruction.points3D
images = read_images_binary(f"{colmap_path}/images.bin")
cameras = reconstruction.cameras

In [2]:
from splat.image import GaussianImage
from splat.utils import (
    build_rotation,
    get_extrinsic_matrix,
    getIntinsicMatrix,
    read_camera_file,
    read_image_file,
)

camera_dict = read_camera_file(colmap_path)
image_dict = read_image_file(colmap_path)
images = {}
for idx in image_dict.keys():
    image = image_dict[idx]
    camera = camera_dict[image.camera_id]
    image = GaussianImage(camera=camera, image=image)
    images[idx] = image


image_num = 100
rotation_matrix = build_rotation(torch.Tensor(image_dict[image_num].qvec).unsqueeze(0))
translation = torch.Tensor(image_dict[image_num].tvec).unsqueeze(0)
extrinsic_matrix = get_extrinsic_matrix(rotation_matrix, translation).T
focal_x, focal_y = camera_dict[image_dict[image_num].camera_id].params[:2]
c_x, c_y = camera_dict[image_dict[image_num].camera_id].params[2:4]
width = camera_dict[image_dict[image_num].camera_id].width
height = camera_dict[image_dict[image_num].camera_id].height
intrinsic_matrix = getIntinsicMatrix(focal_x, focal_y, height, width).T

In [ ]:
from splat.render_engine.utils import compute_fov_from_focal
import math

covariance = gaussians.get_3d_covariance_matrix()[0:1]
mean = gaussians.points[0:1]
t = torch.concat([mean, torch.ones((1, 1))], dim=1)
output = t @ extrinsic_matrix
x = output[:, 0] / output[:, 2]
y = output[:, 1] / output[:, 2]
z = output[:, 2]

print(mean, output, x, y, z)

fovX = compute_fov_from_focal(focal_x, width)
fovY = compute_fov_from_focal(focal_y, height)

tan_fovX = math.tan(fovX / 2)
tan_fovY = math.tan(fovY / 2)

x = torch.clamp(x, -1.3 * tan_fovX, 1.3 * tan_fovX) * output[:, 2]
y = torch.clamp(y, -1.3 * tan_fovY, 1.3 * tan_fovY) * output[:, 2]

j = torch.zeros(1, 3, 3)
j[:, 0, 0] = focal_x / z
j[:, 0, 2] = -(focal_x * x) / (z ** 2)
j[:, 1, 1] = focal_y / z
j[:, 1, 2] = -(focal_y * y) / (z ** 2)

t = extrinsic_matrix[:3, :3].T @ j
print(j)

In [4]:
from splat.render_engine.gaussianScene2 import GaussianScene2

scene = GaussianScene2(gaussians=gaussians)
scene.device = "cpu"
TILE_SIZE = 16

In [ ]:
processed_gaussians = scene.preprocess(
    extrinsic_matrix=extrinsic_matrix,
    intrinsic_matrix=intrinsic_matrix,
    focal_x=focal_x,
    focal_y=focal_y,
    width=width,
    height=height,
    tile_size=TILE_SIZE
)

In [ ]:
processed_gaussians.radius

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(
    processed_gaussians.means_3d[:, 0].detach().cpu(),
    processed_gaussians.means_3d[:, 1].detach().cpu(),
    c=processed_gaussians.color.detach().cpu(),
    s=1,
)

plt.xlim(0, 6000)
plt.ylim(0, 3744)
plt.gca().invert_yaxis()
plt.show()

In [ ]:
from PIL import Image

image_path = "/Users/derek/Desktop/intro_to_gaussian_splatting/data/treehill/images/_DSC8973.JPG"
image = Image.open(image_path)

plt.imshow(image)
plt.show()